In [1]:
import os
import pandas as pd
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# import networkx as nx
import lightgbm as lgb
from math import log
from sklearn.preprocessing import StandardScaler
from submission_functions import make_kaggle_submission, run_experiment, make_kaggle_submissions_from_directory
from categories import monto_columns

origin = pd.read_csv('competencia_01_crudo.csv')

## Funciones

## Data Wranling

In [2]:
foto_mes_array = origin['foto_mes'].to_numpy()
numero_de_cliente_array = origin['numero_de_cliente'].to_numpy()

months = np.sort(np.unique(foto_mes_array))

client_sets_df = pd.DataFrame({
    'month': months,
    'clients': [set(numero_de_cliente_array[foto_mes_array == month]) for month in months]
})

for i in range(len(months[:-2])):
    period_0 = client_sets_df.loc[client_sets_df['month'] == months[i], 'clients'].values[0]
    period_1 = client_sets_df.loc[client_sets_df['month'] == months[i + 1], 'clients'].values[0]
    period_2 = client_sets_df.loc[client_sets_df['month'] == months[i + 2], 'clients'].values[0]
    
    BAJA_2 = period_0 - period_1 - period_2
    BAJA_1 = (period_0 & period_1) - period_2
    
    origin.loc[origin['foto_mes'] == months[i],'clase_ternaria'] = 'CONTINUA'
    origin.loc[(origin['foto_mes'] == months[i]) & (origin['numero_de_cliente'].isin(BAJA_1)), 'clase_ternaria'] = 'BAJA+1'
    origin.loc[(origin['foto_mes'] == months[i]) & (origin['numero_de_cliente'].isin(BAJA_2)), 'clase_ternaria'] = 'BAJA+2'

In [3]:
months = origin.foto_mes.unique()
objective_month = months[-1]
objective_month

target = origin.loc[origin.foto_mes == objective_month]
target

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
817070,249221109,202106,1,0,0,62,305,399.88,20332.88,2115.35,...,0.0,0.0,-20783.64,0.0,8017.0,16701.69,9.0,0.0,1313.76,NaN
817071,249221468,202106,1,0,0,53,42,4708.37,23227.25,5537.14,...,0.0,0.0,-124338.00,0.0,1274.0,81981.26,28.0,0.0,0.00,NaN
817072,249223005,202106,1,0,0,49,207,3534.50,38207.62,534.72,...,0.0,0.0,-28621.20,0.0,3636.0,2669.90,3.0,0.0,18885.30,NaN
817073,249228180,202106,1,0,0,66,325,-138.82,1445.30,417.78,...,0.0,0.0,-17595.00,0.0,9075.0,20496.15,6.0,0.0,1818.15,NaN
817074,249232117,202106,1,0,0,80,378,556.99,36468.62,484.57,...,0.0,0.0,-48423.34,0.0,8757.0,44938.95,7.0,0.0,4058.58,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1598444941,202106,0,0,0,27,1,0.00,0.00,0.00,...,NaN,0.0,NaN,NaN,15.0,NaN,NaN,NaN,0.00,NaN
981942,1598471047,202106,1,0,0,28,1,95.20,95.20,0.00,...,7.0,0.0,NaN,NaN,14.0,NaN,NaN,NaN,0.00,NaN
981943,1598515897,202106,0,0,0,30,1,72.08,72.08,0.00,...,7.0,0.0,NaN,NaN,8.0,NaN,NaN,NaN,0.00,NaN
981944,1598517059,202106,0,0,0,24,1,0.00,0.00,0.00,...,0.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN,0.00,NaN


In [4]:
months

array([202101, 202102, 202103, 202104, 202105, 202106])

#### Dropeamos columnas no útiles

In [5]:
drops = [
        'Master_Fvencimiento',
       'Master_fechaalta', 'Visa_Fvencimiento', 
       'Master_status','Visa_status',
        'Visa_fechaalta', 
        'cprestamos_personales', 'mprestamos_personales',
        ]

origin.drop(columns=drops,inplace=True)

#### Zeros y nulos

In [6]:
mes_train = 202104
mes_score = 202106

train_data = origin[origin['foto_mes'] == mes_train]
score_data = origin[origin['foto_mes'] == mes_score]

train_null_percentage = train_data.isnull().mean() * 100
score_null_percentage = score_data.isnull().mean() * 100

comparison_df = pd.DataFrame({'Train Null Percentage': train_null_percentage, 'Score Null Percentage': score_null_percentage})
comparison_df['diff'] = (comparison_df['Score Null Percentage'] - comparison_df['Train Null Percentage']).abs()

comparison_df_sorted = comparison_df.sort_values('diff', ascending=False)

comparison_df_sorted

,Train Null Percentage,Score Null Percentage,diff
clase_ternaria,0.000000,100.000000,100.000000
Master_mconsumosdolares,59.780608,58.459691,1.320917
Master_mconsumototal,59.780608,58.459691,1.320917
Master_mpagosdolares,59.780608,58.459691,1.320917
Master_mpagospesos,59.780608,58.459691,1.320917
...,...,...,...
cpayroll_trx,0.000000,0.000000,0.000000
mpayroll,0.000000,0.000000,0.000000
mpayroll2,0.000000,0.000000,0.000000
cpayroll2_trx,0.000000,0.000000,0.000000


In [7]:
train_zero_percentage = (train_data == 0).mean() * 100
score_zero_percentage = (score_data == 0).mean() * 100

comparison_df_zero = pd.DataFrame({'Train Zero Percentage': train_zero_percentage, 'Score Zero Percentage': score_zero_percentage})

comparison_df_zero['diff_zero_percentage'] = (comparison_df_zero['Score Zero Percentage'] - comparison_df_zero['Train Zero Percentage']).abs()
diff_zero_percentage_sorted = comparison_df_zero.sort_values('diff_zero_percentage',ascending=False)

diff_zero_percentage_sorted

,Train Zero Percentage,Score Zero Percentage,diff_zero_percentage
Master_fultimo_cierre,0.000000,68.891773,68.891773
Visa_fultimo_cierre,0.000000,68.829909,68.829909
cmobile_app_trx,28.197331,23.683859,4.513471
mcuenta_corriente,47.169236,49.692496,2.523260
ctransferencias_recibidas,26.395880,24.303719,2.092161
...,...,...,...
tcuentas,0.000000,0.000000,0.000000
cproductos,0.000000,0.000000,0.000000
cliente_antiguedad,0.000000,0.000000,0.000000
cliente_edad,0.000000,0.000000,0.000000


##### Dropeamos ['Master_fultimo_cierre', 'Visa_fultimo_cierre']

In [8]:
origin.drop(columns=['Master_fultimo_cierre', 'Visa_fultimo_cierre'],inplace=True)

#### Drifting

In [9]:
def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
    breakpoints = sorted(list(set(bin_edges2)))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null

In [10]:
psi_results = []
for column in train_data.columns:
  if column not in ['foto_mes', 'clase_ternaria']:
    train_variable = train_data[column]
    score_variable = score_data[column]
    psi_value = psi(train_variable, score_variable)
    psi_results.append({'feature': column, 'psi': psi_value})

psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)


/var/folders/c9/qzlbnk_51hs724bv_4b6wcxm0000gn/T/ipykernel_95442/3695061145.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
/var/folders/c9/qzlbnk_51hs724bv_4b6wcxm0000gn/T/ipykernel_95442/3695061145.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)


In [11]:
psi_df.head(50)

,feature,psi
128,Visa_Finiciomora,inf
109,Master_Finiciomora,inf
118,Master_fultimo_cierre,1.129066
137,Visa_fultimo_cierre,0.937571
48,cpayroll_trx,0.241274
49,mpayroll,0.213187
69,mcomisiones_mantenimiento,0.051215
78,mtransferencias_recibidas,0.034650
77,ctransferencias_recibidas,0.021754
21,mcuentas_saldo,0.017142


### Normatilzation

In [12]:
# monto_columns_to_log = [i for i in monto_columns if i in origin.columns]
# origin['minversion1_pesos'].apply(lambda x: np.log(x) if x > 0 else np.nan).plot(kind='hist',bins=100)

In [13]:
# for i in monto_columns_to_log:
#     origin[i] = origin[i].apply(lambda x: np.log(x) if x > 0 else np.nan)

### Lags & Deltas

In [14]:
### Lags

origin.sort_values(by=['numero_de_cliente', 'foto_mes'],inplace=True)

not_lags = [
        'cliente_vip', 'internet',
        'cliente_antiguedad', 'cliente_edad', 'clase_ternaria'
    ]    
columns_to_lag = [i for i in origin.columns if i not in not_lags]

lags = [1, 2]  # Lag periods
for lag in lags:
    for i in columns_to_lag:
        origin[f'{i}_lag_{lag}'] = origin.groupby('numero_de_cliente')[i].shift(lag)


/var/folders/c9/qzlbnk_51hs724bv_4b6wcxm0000gn/T/ipykernel_95442/1814061146.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  origin[f'{i}_lag_{lag}'] = origin.groupby('numero_de_cliente')[i].shift(lag)
/var/folders/c9/qzlbnk_51hs724bv_4b6wcxm0000gn/T/ipykernel_95442/1814061146.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  origin[f'{i}_lag_{lag}'] = origin.groupby('numero_de_cliente')[i].shift(lag)
/var/folders/c9/qzlbnk_51hs724bv_4b6wcxm0000gn/T/ipykernel_95442/1814061146.py:14: PerformanceWarning: DataFrame is highly

In [15]:
### Deltas

columns_to_delta = columns_to_lag
lags = [1, 2]  # Lag periods

for col in columns_to_delta:
    for lag in lags:
        # Delta between current and lag
        origin[f'delta_{col}_{lag}'] = origin[col] - origin[f'{col}_lag_{lag}']
    
    # Delta between consecutive lags (1-period lag and 2-period lag)
    for i in range(1, len(lags)):
        origin[f'delta_{col}_{lags[i-1]}_{lags[i]}'] = origin[f'{col}_lag_{lags[i-1]}'] - origin[f'{col}_lag_{lags[i]}']


/var/folders/c9/qzlbnk_51hs724bv_4b6wcxm0000gn/T/ipykernel_95442/3356127686.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  origin[f'delta_{col}_{lag}'] = origin[col] - origin[f'{col}_lag_{lag}']
/var/folders/c9/qzlbnk_51hs724bv_4b6wcxm0000gn/T/ipykernel_95442/3356127686.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  origin[f'delta_{col}_{lag}'] = origin[col] - origin[f'{col}_lag_{lag}']
/var/folders/c9/qzlbnk_51hs724bv_4b6wcxm0000gn/T/ipykernel_95442/3356127686.py:13: PerformanceWarning: DataFrame is highly fragmented.

### Tarjetas

In [16]:
card_limits = [   
    'Master_mfinanciacion_limite', 'Visa_mfinanciacion_limite',
    'Master_mlimitecompra', 'Visa_mlimitecompra',
]
origin[card_limits].notna().sum() / len(origin[card_limits])

Master_mfinanciacion_limite    0.896818
Visa_mfinanciacion_limite      0.950264
Master_mlimitecompra           0.896818
Visa_mlimitecompra             0.950264
dtype: float64

In [17]:
bajas = origin.loc[origin.clase_ternaria != 'CONTINUA']

np.sum((bajas['Master_mlimitecompra'].notna() == False) & (bajas['Visa_mlimitecompra'].notna() == False)) / len(bajas)

np.float64(0.04507069010041764)

In [18]:
len(origin.loc[(origin['Master_mlimitecompra'].isna() & origin['Visa_mlimitecompra'].isna()) & (origin.foto_mes == objective_month)]) / len(origin.loc[origin.foto_mes == objective_month])

0.04150998326014702

In [19]:
labels = [f'tag_{i}' for i in range(1,17)]

limits_to_label = origin['Visa_mlimitecompra'].combine_first(origin['Master_mlimitecompra'])

cardlimits_labels = pd.qcut(
    limits_to_label.apply(lambda x: log(x)),q=20,
    duplicates='drop',
    labels=labels
)
cardlimits_labels

0         tag_12
162026    tag_13
324672    tag_13
488357    tag_13
652447    tag_13
           ...  
981941     tag_9
981942    tag_13
981943    tag_13
981944    tag_13
981945     tag_1
Name: Visa_mlimitecompra, Length: 981946, dtype: category
Categories (16, object): ['tag_1' < 'tag_2' < 'tag_3' < 'tag_4' ... 'tag_13' < 'tag_14' < 'tag_15' < 'tag_16']

In [20]:
cardlimits_labels[cardlimits_labels.isna()]

817093    NaN
45        NaN
162070    NaN
324716    NaN
488401    NaN
         ... 
652293    NaN
652373    NaN
652399    NaN
652414    NaN
652416    NaN
Name: Visa_mlimitecompra, Length: 40465, dtype: category
Categories (16, object): ['tag_1' < 'tag_2' < 'tag_3' < 'tag_4' ... 'tag_13' < 'tag_14' < 'tag_15' < 'tag_16']

In [21]:
"""'ctarjeta_visa',
 'ctarjeta_visa_transacciones',
 'mtarjeta_visa_consumo',
 'ctarjeta_master',
 'ctarjeta_master_transacciones',
 'mtarjeta_master_consumo',"""

"'ctarjeta_visa',\n 'ctarjeta_visa_transacciones',\n 'mtarjeta_visa_consumo',\n 'ctarjeta_master',\n 'ctarjeta_master_transacciones',\n 'mtarjeta_master_consumo',"

## Correr experimento

In [22]:
experiment_name = 'RU004-1'
origin.to_csv(f'experiment_{experiment_name}.csv')

In [23]:
PARAM = {
    "experimento":experiment_name,
    "semilla_primigenia":849991,
    "input": {
        "dataset":f"/Users/nahuelalejocaceres/Desktop/Maestría/DMEyF/Number_1/experiment_{experiment_name}.csv",
        "training":[202103,202104],
        "future":[202106],
    }
    
}
run_experiment(PARAM)

/Users/nahuelalejocaceres/Desktop/Maestría/DMEyF/Number_1/submission_functions.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset["clase01"] = np.where(dataset["clase_ternaria"].isin(["BAJA+2", "BAJA+1"]), 1, 0)
/Users/nahuelalejocaceres/Desktop/Maestría/DMEyF/Number_1/submission_functions.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset["train"] = 0
/Users/nahuelalejocaceres/Desktop/Maestría/DMEyF/Number_1/.venv/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in 

[LightGBM] [Info] Number of positive: 4172, number of negative: 323603
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.176941 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 115418
[LightGBM] [Info] Number of data points in the train set: 327775, number of used features: 844
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.012728 -> initscore=-4.351122
[LightGBM] [Info] Start training from score -4.351122


La generación de los archivos para Kaggle ha terminado



In [24]:
os.chdir('..')

### Subir tiradas a kaggle

In [25]:
make_kaggle_submissions_from_directory(
    directory=experiment_name,
    competition_name='dm-ey-f-2024-primera',
    message='10k iterations'
)

Submitting RU004-1_12500.csv to competition 'dm-ey-f-2024-primera'


100%|██████████| 1.93M/1.93M [00:02<00:00, 878kB/s] 


Successfully submitted to DMEyF 2024 PrimeraSubmission of 'RU004-1_12500.csv' completed with message: 10k iterations
Submitting RU004-1_14000.csv to competition 'dm-ey-f-2024-primera'


100%|██████████| 1.93M/1.93M [00:02<00:00, 884kB/s] 


Successfully submitted to DMEyF 2024 PrimeraSubmission of 'RU004-1_14000.csv' completed with message: 10k iterations
Submitting RU004-1_10500.csv to competition 'dm-ey-f-2024-primera'


...

Successfully submitted to DMEyF 2024 PrimeraSubmission of 'RU004-1_11500.csv' completed with message: 10k iterations
Submitting RU004-1_11000.csv to competition 'dm-ey-f-2024-primera'


100%|██████████| 1.93M/1.93M [00:01<00:00, 1.12MB/s]


Successfully submitted to DMEyF 2024 PrimeraSubmission of 'RU004-1_11000.csv' completed with message: 10k iterations
Submitting RU004-1_13000.csv to competition 'dm-ey-f-2024-primera'


100%|██████████| 1.93M/1.93M [00:02<00:00, 887kB/s] 


Successfully submitted to DMEyF 2024 PrimeraSubmission of 'RU004-1_13000.csv' completed with message: 10k iterations

All CSV files in 'RU004-1' have been submitted to the competition 'dm-ey-f-2024-primera'.


In [26]:
origin.to_csv(f'experiment_{experiment_name}.csv')